In [170]:
colors = {
    'white': 0,
    'black': 1,
}
figures = {
    'Pawn': 1,
    'Rook': 2,
    'Knight': 3,
    'Bishop': 4,
    'Queen': 5,
    'King': 6,
}
reverse_figures = {v: k[0] for k, v in figures.items()}

In [171]:
import copy

In [172]:
class Piece:
    def __init__(self, color, position):
        self.color = color
        self.position = position
        self.eliminated = False
        self.destructable = True

    def eliminate(self):
        self.eliminated = True

In [173]:
class Pawn(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.first_move = True
        self.figure = figures['Pawn']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        if self.color == colors['white']:
            if row == 1 and board_instance.check_for_empty((row + 2, col)) and board_instance.check_for_empty((row + 1, col)):
                moves.append((row + 2, col))
            if row + 1 < 8 and board_instance.check_for_empty((row + 1, col)):
                moves.append((row + 1, col))
            dc = [[1, 1], [1, -1]]
            for i in range(2):
                new_row, new_col = row + dc[i][0], col + dc[i][1]
                if 0 <= new_row < 8 and 0 <= new_col < 8 and board_instance.check_for_enemy((new_row, new_col), self.color):
                    moves.append((new_row, new_col))

        if self.color == colors['black']:
            if row == 6 and board_instance.check_for_empty((row - 2, col)) and board_instance.check_for_empty((row - 1, col)):
                moves.append((row - 2, col))
            if row - 1 >= 0 and board_instance.check_for_empty((row - 1, col)):
                moves.append((row - 1, col))
            dc = [[-1, 1], [-1, -1]]
            for i in range(2):
                new_row, new_col = row + dc[i][0], col + dc[i][1]
                if 0 <= new_row < 8 and 0 <= new_col < 8 and board_instance.check_for_enemy((new_row, new_col), self.color):
                    moves.append((new_row, new_col))

        return moves

In [174]:
class Rook(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.first_move = True
        self.figure = figures['Rook']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        directions = [[1, 0], [-1, 0], [0, 1], [0, -1]]

        for dr,dc in directions:
            r,c = row +dr , col + dc
            while 0 <= r < 8 and 0 <= c < 8:

                if board_instance.check_for_friendly((r,c), self.color):
                    break
                moves.append((r,c))
                if board_instance.check_for_enemy((r,c), self.color):
                    break
                r += dr
                c += dc

        return moves


In [175]:
class Knight(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['Knight']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        dc = [[1, 2], [1, -2], [-1, 2], [-1, -2], [2, 1], [2, -1], [-2, 1], [-2, -1]]

        for i in range(8):
            new_row, new_col = row + dc[i][0], col + dc[i][1]
            if 0 <= new_row < 8 and 0 <= new_col < 8 and not board_instance.check_for_friendly((new_row, new_col), self.color):
                moves.append((new_row, new_col))

        return moves

In [176]:
class Bishop(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['Bishop']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        directions = [[1, 1], [1, -1], [-1, 1], [-1, -1]]

        for dr, dc in directions:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if board_instance.check_for_friendly((r, c), self.color):
                    break
                moves.append((r, c))
                if board_instance.check_for_enemy((r, c), self.color):
                    break
                r += dr
                c += dc

        return moves

In [177]:
class Queen(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['Queen']

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        directions = [[1, 0], [-1, 0], [0, 1], [0, -1], [1, 1], [1, -1], [-1, 1], [-1, -1]]

        for dr, dc in directions:
            r, c = row + dr, col + dc
            while 0 <= r < 8 and 0 <= c < 8:
                if board_instance.check_for_friendly((r, c), self.color):
                    break
                moves.append((r, c))
                if board_instance.check_for_enemy((r, c), self.color):
                    break
                r += dr
                c += dc

        return moves



In [178]:
class King(Piece):
    def __init__(self, color, position):
        super().__init__(color, position)
        self.figure = figures['King']
        self.destructable = False

    def valid_moves(self, board_instance):
        moves = []
        row, col = self.position

        dc = [[1, 0], [-1, 0], [0, 1], [0, -1], [1, 1], [1, -1], [-1, 1], [-1, -1]]

        for i in range(8):
            new_row, new_col = row + dc[i][0], col + dc[i][1]
            if 0 <= new_row < 8 and 0 <= new_col < 8 and not board_instance.check_for_friendly((new_row, new_col), self.color):
                moves.append((new_row, new_col))

        # restricted moves
        enemy_moves = board_instance.enemy_moves(self.color)
        king_position = board_instance.get_enemy_king_position(self.color)
        if king_position:
            restricted_positions = []
            king_dc = [[1, 0], [-1, 0], [0, 1], [0, -1], [1, 1], [1, -1], [-1, 1], [-1, -1]]
            king_row, king_col = king_position
            for dr, dc in king_dc:
                new_row, new_col = king_row + dr, king_col + dc
                if 0 <= new_row < 8 and 0 <= new_col < 8:
                    restricted_positions.append((new_row, new_col))

            moves = [move for move in moves if move not in restricted_positions]
        moves = [move for move in moves if move not in enemy_moves]

        valid_moves = []
        for move in moves:
            future_board = copy.deepcopy(board_instance)
            future_board.board[move[0]][move[1]] = self
            future_board.board[row][col] = None
            future_enemy_moves = future_board.future_enemy_moves(self.color)
            if move not in future_enemy_moves:
                valid_moves.append(move)

        return valid_moves

In [179]:
class Board:
    def __init__(self):
        self.board = self.create_board()
        self.populate_board()
        self.captured_pieces = []

    def print_color_from_number(self, number):
        for color, num in colors.items():
            if num == number:
                print(color)
                return
        print("Color not found")

    def create_board(self):
        return [[None for _ in range(8)] for _ in range(8)]

    def populate_board(self):
        self.place_pieces(Pawn, 1, colors['white'])
        self.place_pieces(Pawn, 6, colors['black'])
        self.place_pieces(Rook, 0, colors['white'], [0, 7])
        self.place_pieces(Rook, 7, colors['black'], [0, 7])
        self.place_pieces(Knight, 0, colors['white'], [1, 6])
        self.place_pieces(Knight, 7, colors['black'], [1, 6])
        self.place_pieces(Bishop, 0, colors['white'], [2, 5])
        self.place_pieces(Bishop, 7, colors['black'], [2, 5])
        self.place_pieces(Queen, 0, colors['white'], [3])
        self.place_pieces(Queen, 7, colors['black'], [3])
        self.place_pieces(King, 0, colors['white'], [4])
        self.place_pieces(King, 7, colors['black'], [4])

    def place_pieces(self, piece_class, row, color, cols=None):
        if cols is None:
            for col in range(8):
                self.board[row][col] = piece_class(color, (row, col))
        else:
            for col in cols:
                self.board[row][col] = piece_class(color, (row, col))

    def move_piece(self, start_pos, end_pos):
        piece = self.get_piece(start_pos)
        if not piece:
            return False
        if end_pos not in piece.valid_moves(self):
            return False
        target_piece = self.get_piece(end_pos)
        if target_piece:
            if isinstance(target_piece, King):
                return False
            if target_piece.color != piece.color:
                target_piece.eliminate()
                self.captured_pieces.append(target_piece)

        self.board[start_pos[0]][start_pos[1]] = None
        piece.position = end_pos
        self.board[end_pos[0]][end_pos[1]] = piece
        return True

    def show_board(self):
        print("  " + " ".join(str(x) for x in range(8)))
        for y, row in enumerate(self.board):
            print(f"{y} " + " ".join([reverse_figures[piece.figure] if piece else '.' for piece in row]))

    def get_valid_moves(self, position):
        piece = self.get_piece(position)
        if piece:
            return piece.valid_moves(self)
        return []

    def get_piece(self, position):
        row, col = position
        return self.board[row][col]

    def check_for_enemy(self, position, color):
        piece = self.get_piece(position)
        return piece and piece.color != color

    def check_for_friendly(self, position, color):
        piece = self.get_piece(position)
        return piece and piece.color == color

    def check_for_empty(self, position):
        return not self.get_piece(position)

    def enemy_moves(self, color):
        moves = []
        for row in range(8):
            for col in range(8):
                piece = self.get_piece((row, col))
                if piece and piece.color != color and not isinstance(piece, King):
                    piece_moves = piece.valid_moves(self)
                    if isinstance(piece, Pawn):
                        piece_moves = self.filter_pawn_moves(piece, piece_moves, row, col)
                    moves.extend(piece_moves)
        return moves

    def filter_pawn_moves(self, piece, piece_moves, row, col):
        if piece.color == colors['white']:
            return [move for move in piece_moves if not (move[0] == row + 1 and move[1] == col) and not (move[0] == row + 2 and move[1] == col)]
        elif piece.color == colors['black']:
            return [move for move in piece_moves if not (move[0] == row - 1 and move[1] == col) and not (move[0] == row - 2 and move[1] == col)]
        return piece_moves

    def future_enemy_moves(self, color):
        return self.enemy_moves(color)

    def get_enemy_king_position(self, color):
        for row in range(8):
            for col in range(8):
                piece = self.get_piece((row, col))
                if piece and isinstance(piece, King) and piece.color != color:
                    return (row, col)
        return None

    def enemy_king_moves(self, color):
        enemy_king_position = self.get_enemy_king_position(color)
        if enemy_king_position:
            enemy_king = self.get_piece(enemy_king_position)
            if isinstance(enemy_king, King):
                return enemy_king.valid_moves(self)
        return []

    def get_king_position(self, color):
        for row in range(8):
            for col in range(8):
                piece = self.get_piece((row, col))
                if piece and isinstance(piece, King) and piece.color == color:
                    return (row, col)
        return None

    def is_check(self, color):
        enemy_king = self.get_enemy_king_position(color)
        checking_pieces=[]
        for row in range(8):
            for col in range(8):
                piece = self.get_piece((row, col))
                if piece and piece.color == color:
                    piece_moves = piece.valid_moves(self)
                    if isinstance(piece, Pawn):
                        piece_moves = self.filter_pawn_moves(piece, piece_moves, row, col)
                    if enemy_king in piece_moves:
                        checking_pieces.append(piece)

        return bool(checking_pieces), checking_pieces







In [180]:
board = Board()
board.populate_board()

In [181]:
import pygame
import sys

In [182]:
import pygame
import sys

# Initialize Pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 800
ROWS, COLS = 8, 8
SQUARE_SIZE = WIDTH // COLS

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
LIGHT_BROWN = (240, 217, 181)
DARK_BROWN = (181, 136, 99)
BLUE = (0, 0, 255)
RED = (255, 0, 0)

WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Chess')

def load_images():
    pieces = {
        'wp': 'white pawn',
        'bp': 'black pawn',
        'wr': 'white rook',
        'br': 'black rook',
        'wn': 'white knight',
        'bn': 'black knight',
        'wb': 'white bishop',
        'bb': 'black bishop',
        'wq': 'white queen',
        'bq': 'black queen',
        'wk': 'white king',
        'bk': 'black king'
    }
    images = {}
    for piece, name in pieces.items():
        file_name = name.replace(' ', '_') + '.png'
        try:
            images[piece] = pygame.transform.scale(pygame.image.load(f'images/{file_name}'), (SQUARE_SIZE, SQUARE_SIZE))
        except FileNotFoundError:
            print(f"File not found: images/{file_name}")
    return images

def draw_board(win):
    win.fill(WHITE)
    for row in range(ROWS):
        for col in range(COLS):
            color = LIGHT_BROWN if (row + col) % 2 == 0 else DARK_BROWN
            pygame.draw.rect(win, color, (col * SQUARE_SIZE, row * SQUARE_SIZE, SQUARE_SIZE, SQUARE_SIZE))

def draw_pieces(win, board):
    for row in range(ROWS):
        for col in range(COLS):
            piece = board[row][col]
            if piece:
                piece_key = f"{'w' if piece.color == 0 else 'b'}{'n' if piece.__class__.__name__ == 'Knight' else piece.__class__.__name__[0].lower()}"
                win.blit(IMAGES[piece_key], (col * SQUARE_SIZE, row * SQUARE_SIZE))

def highlight_moves(win, moves):
    for move in moves:
        row, col = move
        pygame.draw.circle(win, BLUE, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), SQUARE_SIZE // 4)

def highlight_check(win, board_instance, color):
    is_check, checking_pieces = board_instance.is_check(color)
    if is_check:
        enemy_king_position = board_instance.get_enemy_king_position(color)
        if enemy_king_position:
            row, col = enemy_king_position
            pygame.draw.circle(win, RED, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), SQUARE_SIZE // 2, 5)
            for piece in checking_pieces:
                row, col = piece.position
                pygame.draw.circle(win, RED, (col * SQUARE_SIZE + SQUARE_SIZE // 2, row * SQUARE_SIZE + SQUARE_SIZE // 2), SQUARE_SIZE // 2, 5)

def handle_mouse_button_up(board_instance, selected_piece, valid_moves, dragging, SQUARE_SIZE):
    if dragging:
        pos = pygame.mouse.get_pos()
        new_col, new_row = pos[0] // SQUARE_SIZE, pos[1] // SQUARE_SIZE
        if (new_row, new_col) in valid_moves:
            if isinstance(board_instance.get_piece((new_row, new_col)), King):
                dragging = False
                return selected_piece, valid_moves, dragging
            board_instance.board[selected_piece.position[0]][selected_piece.position[1]] = None
            selected_piece.position = (new_row, new_col)
            board_instance.board[new_row][new_col] = selected_piece

def handle_mouse_button_down(board_instance, SQUARE_SIZE):
    pos = pygame.mouse.get_pos()
    col, row = pos[0] // SQUARE_SIZE, pos[1] // SQUARE_SIZE
    selected_piece = board_instance.get_piece((row, col))
    valid_moves = []
    dragging = False
    drag_offset_x = 0
    drag_offset_y = 0
    if selected_piece:
        valid_moves = board_instance.get_valid_moves((row, col))
        dragging = True
        drag_offset_x = pos[0] - col * SQUARE_SIZE
        drag_offset_y = pos[1] - row * SQUARE_SIZE
    return selected_piece, valid_moves, dragging, drag_offset_x, drag_offset_y

def handle_mouse_motion(dragging, SQUARE_SIZE):
    col, row = -1, -1  # Initialize with default values
    if dragging:
        pos = pygame.mouse.get_pos()
        col, row = pos[0] // SQUARE_SIZE, pos[1] // SQUARE_SIZE
    return col, row

IMAGES = load_images()

import pygame
import sys

def main():
    board_instance = board

    selected_piece = None
    valid_moves = []
    dragging = False
    drag_offset_x = 0
    drag_offset_y = 0

    run = True
    while run:
        event = pygame.event.wait()

        if event.type == pygame.QUIT:
            run = False

        elif event.type == pygame.MOUSEBUTTONDOWN:
            selected_piece, valid_moves, dragging, drag_offset_x, drag_offset_y = handle_mouse_button_down(board_instance, SQUARE_SIZE)

        elif event.type == pygame.MOUSEBUTTONUP:
            handle_mouse_button_up(board_instance, selected_piece, valid_moves, dragging, SQUARE_SIZE)
            if selected_piece:
                is_check = board_instance.is_check(selected_piece.color)
                print(f"Is check: {is_check}")

            dragging = False
            selected_piece = None
            valid_moves = []

        elif event.type == pygame.MOUSEMOTION:
            col, row = handle_mouse_motion(dragging, SQUARE_SIZE)

        draw_board(WIN)
        draw_pieces(WIN, board_instance.board)
        highlight_moves(WIN, valid_moves)

        if dragging and selected_piece:
            pos = pygame.mouse.get_pos()
            WIN.blit(IMAGES[f"{'w' if selected_piece.color == 0 else 'b'}{'n' if selected_piece.__class__.__name__ == 'Knight' else selected_piece.__class__.__name__[0].lower()}"], (pos[0] - drag_offset_x, pos[1] - drag_offset_y))
        pygame.display.flip()

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main()

Is check: (False, [])
Is check: (False, [])
Is check: (False, [])
Is check: (False, [])
Is check: (False, [])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (True, [<__main__.Queen object at 0x00000286F8608EF0>])
Is check: (False, [])
Is check: (False, [])
Is check: (False, [])
Is check: (True, [<__main__.Knight object at 0x00000286F860B020>])
Is check: (True, [<__main__.Knight object at 0x00000286F860B020>, <__main__.Bishop object at 0x00

SystemExit: 